Verify Python environment

In [1]:
import sys
print(sys.executable)

c:\Users\alals\repos\perf-pilot\venv\Scripts\python.exe


List installed packages

In [ ]:
%pip list

Import dependencies

In [67]:
import requests
from pprint import pprint
import csv
from datetime import datetime
from dateutil.relativedelta import relativedelta
import time
import pandas as pd
import ast
from collections import Counter

API preparations

In [68]:
# Bugzilla API key
API_KEY = "sicSwVcOxOwvLCn12804YSB0nhlCuSBw4FkhKoKK"
BUGZILLA_API_URL = "https://bugzilla.mozilla.org/rest"


Get performance bugs that are also regressions and are created after a certain time

In [9]:
ENDPOINT = f"{BUGZILLA_API_URL}/bug"

FIELDS = ["id", "regressed_by", "product", "component", "creation_time"]

now = datetime.now()
threshold_time = now - relativedelta(days=17)
print(threshold_time)

params = {
        "api_key": API_KEY,
        "limit": 5,
        "v1": threshold_time,
        "o1": "greaterthan",
        "f1": "creation_ts",
        "include_fields": ",".join(FIELDS),
        "offset": 1
    }
    
response = requests.get(ENDPOINT, params=params)

response.json()

# bugs = response.json().get("bugs", [])

# for bug in bugs:
#             print(f"#{bug['id']}:")
#             print(f"  Regressed by: {bug['regressed_by']}")
#             print()
    

2025-03-14 13:43:51.886733


{'bugs': [{'id': 1954122,
   'regressed_by': [],
   'product': 'Firefox',
   'component': 'New Tab Page',
   'creation_time': '2025-03-14T13:47:11Z'},
  {'regressed_by': [],
   'id': 1954123,
   'product': 'Firefox',
   'component': 'Untriaged',
   'creation_time': '2025-03-14T13:47:39Z'},
  {'product': 'Testing',
   'regressed_by': [],
   'id': 1954124,
   'creation_time': '2025-03-14T13:49:45Z',
   'component': 'Performance'},
  {'component': 'Privacy: Anti-Tracking',
   'creation_time': '2025-03-14T13:52:27Z',
   'regressed_by': [],
   'id': 1954125,
   'product': 'Core'},
  {'product': 'Firefox',
   'regressed_by': [],
   'id': 1954127,
   'creation_time': '2025-03-14T13:52:44Z',
   'component': 'Tabbed Browser'}]}

Get bugs and save them to a csv file

In [27]:
BUGZILLA_API = f"{BUGZILLA_API_URL}/bug"
FIELDS = ["id", "summary", "regressed_by"]
LIMIT = 100  # Page size
MAX_TOTAL = 1000  # Cap on total bugs to fetch

def fetch_all_regressed_bugs():
    offset = 0
    all_bugs = []

    while len(all_bugs) < MAX_TOTAL:
        remaining = MAX_TOTAL - len(all_bugs)
        page_limit = min(LIMIT, remaining)

        params = {
            "api_key": API_KEY,
            "include_fields": ",".join(FIELDS),
            "f1": "regressed_by",
            "o1": "isnotempty",
            "limit": page_limit,
            "offset": offset
        }

        response = requests.get(BUGZILLA_API, params=params)
        if response.status_code != 200:
            print(f"Failed to fetch bugs: {response.status_code}")
            print(response.text)
            break

        bugs = response.json().get("bugs", [])
        if not bugs:
            break  # No more bugs

        all_bugs.extend(bugs)
        offset += page_limit

    print(f"✅ Total bugs fetched: {len(all_bugs)}")
    return all_bugs

def save_bugs_to_csv(bugs, filename="regressed_bugs.csv"):
    with open(filename, mode="w", newline="", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(FIELDS)  # Header

        for bug in bugs:
            writer.writerow([
                bug.get("id"),
                bug.get("summary"),
                ",".join(str(x) for x in bug.get("regressed_by", []))
            ])

    print(f"📁 Bugs saved to {filename}")



bugs = fetch_all_regressed_bugs()
save_bugs_to_csv(bugs)


✅ Total bugs fetched: 1000
📁 Bugs saved to regressed_bugs.csv


Get bug ticket attachments for a single bug

In [30]:
ENDPOINT = "https://bugzilla.mozilla.org/rest/bug/1947908/attachment"

FIELDS = ["id", "regressed_by"]

params = {
        "api_key": API_KEY,
        "keywords": ["perf"],
        "limit": 5,
        "f1": "regressed_by",
        "o1": "isnotempty",
        "include_fields": ",".join(FIELDS),
        "offset": 1
    }
    
response = requests.get(ENDPOINT)

pprint(response.json())

{'attachments': {},
 'bugs': {'1947908': [{'attacher': 'brennie@mozilla.com',
                       'bug_id': 1947908,
                       'content_type': 'text/x-phabricator-request',
                       'creation_time': '2025-03-05T05:50:32Z',
                       'creator': 'brennie@mozilla.com',
                       'creator_detail': {'email': 'brennie@mozilla.com',
                                          'id': 529428,
                                          'name': 'brennie@mozilla.com',
                                          'nick': 'beth',
                                          'real_name': 'Beth Rennie [:beth] '
                                                       '(she/her)'},
                       'data': 'aHR0cHM6Ly9waGFicmljYXRvci5zZXJ2aWNlcy5tb3ppbGxhLmNvbS9EMjQwNDE4',
                       'description': 'Bug 1947908 - Remove update:slug event '
                                      'from ExperimentStore '
                                      'r?

- from regressions.csv taken from treeherder module, get products and components that are important for performance
- Get all the bugs in the last year that have the same product and components from above
- For each regression bug, inspect its "regressed by" field and create a new list of bugs called "regressor_bugs"
- For each bug in the last year, if the bug also exists in regressor_bugs.csv, mark the "caused perf regression" as true
- For each bug/ticket in the last year, have these columns bug id, summary, caused perf regression, product, component and name this list bugs.csv

In [45]:
BUGZILLA_API = f"{BUGZILLA_API_URL}/bug"
FIELDS = ["id", "summary", "regressed_by", "product", "component", "creation_time"]
LIMIT = 150  # Page size

TIMESPAN_IN_DAYS = 365

offset = 0
all_bugs_list = []

now = datetime.now()
threshold_time = now - relativedelta(days=TIMESPAN_IN_DAYS)

# get all the bugs in the last year
while True:

    params = {
        "api_key": API_KEY,
        "include_fields": ",".join(FIELDS),
        "v1": threshold_time,
        "o1": "greaterthan",
        "f1": "creation_ts",
        "limit": LIMIT,
        "offset": offset
    }

    response = requests.get(BUGZILLA_API, params=params)

    bugs = response.json().get("bugs", [])
    if not bugs:
        break  # No more bugs

    all_bugs_list.extend(bugs)
    offset += LIMIT

all_bugs_df = pd.DataFrame(all_bugs_list)
all_bugs_df.to_csv("../datasets/all_bugs.csv", index=False)


In [69]:
all_bugs_df = pd.read_csv("../datasets/all_bugs.csv")
all_bugs_df['creation_time'] = pd.to_datetime(all_bugs_df['creation_time'])
all_bugs_df['regressed_by'] = all_bugs_df['regressed_by'].apply(ast.literal_eval)
all_bugs_df_sorted = all_bugs_df.sort_values(by='creation_time')

# sorted list of bugs
all_bugs_list = all_bugs_df.to_dict(orient='records')

# get all regressions from treeherder module
regressions_df = pd.read_csv("../datasets/regressions.csv")
regressions_df['reg_perf_tests_list'] = regressions_df['reg_perf_tests_list'].apply(ast.literal_eval)
regressions_list = regressions_df.to_dict(orient='records')

# convert regressions_list from list to dict for fast lookup
regressions_dict = {}
for bug in regressions_list:
    regressions_dict[bug['regression_bug_id']] = bug

# convert all_bugs from list to dict for fast lookup
all_bugs_dict = {}
for bug in all_bugs_list:
    all_bugs_dict[bug['id']] = bug

# get regressor bugs from regression bugs and add their relevant info
all_regressor_bugs_list = []
regression_bugs_not_in_bugzilla_bugs = []
regressions_without_regressor = []

for bug in regressions_list:

    regression_bug_id = bug['regression_bug_id']
    regression_bug = all_bugs_dict.get(regression_bug_id)

    if not regression_bug:
        regression_bugs_not_in_bugzilla_bugs.append(regression_bug_id)
        # pprint(bug['perf_reg_alert_summary_id'])
        continue

    regressor_bug_ids_list = regression_bug.get('regressed_by')
    if not regressor_bug_ids_list:
        regressions_without_regressor.append(regression_bug_id)
        continue

    for bug_id in regressor_bug_ids_list:
        regressor_bug = {"bug_id": bug_id, 
                            "regressed_perf_tests": bug['reg_perf_tests_list'], 
                            "regressed_product": regression_bug['product'], 
                            "regressed_component": regression_bug['component']}

        all_regressor_bugs_list.append(regressor_bug)

# pprint("regression_bugs_not_in_bugzilla_bugs:\n")
# pprint(regression_bugs_not_in_bugzilla_bugs)
# print()

# gather relevant components and products
relevant_components = set()
relevant_products = set()

# for counting purposes
relevant_components_list = []
relevant_products_list = []

for bug in all_regressor_bugs_list:
    product = bug.get("regressed_product")
    component = bug.get("regressed_component")

    relevant_products.add(product)
    relevant_components.add(component)

    relevant_components_list.append(component)
    relevant_products_list.append(product)


products_count = Counter(relevant_products_list)
components_count = Counter(relevant_components_list)

pprint(products_count)
pprint(components_count)


# convert all_regressor_bugs_list from list to dict for fast lookup
all_regressor_bugs_dict = {}
for bug in all_regressor_bugs_list:
    all_regressor_bugs_dict[bug['bug_id']] = bug

# create the final csv file
all_bugs_with_added_regressor_info_list = []
for bug in all_bugs_list:
    if (bug['component'] not in relevant_components) or (bug['product'] not in relevant_products):
        continue

    bug_id = bug['id']
    bug_summary = bug.get('summary')
    bug_component = bug.get('component')
    bug_product = bug.get('product')
    bug_creation_time = bug.get('creation_time')

    bug_is_perf_regressor = bug_id in all_regressor_bugs_dict
    bug_is_perf_regression = bug_id in regressions_dict

    regressed_perf_tests_list = []
    perf_reg_alert_summary_id = 0

    if bug_is_perf_regressor:
        bug_with_regressed_perf_tests = all_regressor_bugs_dict.get(bug_id)
        regressed_perf_tests_list = bug_with_regressed_perf_tests.get('regressed_perf_tests')

    if bug_is_perf_regression:
        regression_bug = regressions_dict.get(bug_id)
        perf_reg_alert_summary_id = regression_bug.get('perf_reg_alert_summary_id')

    
    bug_with_added_info = {'bug_id': bug_id, 
                           'bug_summary': bug_summary, 
                           'bug_is_perf_regressor': bug_is_perf_regressor,
                           'bug_is_perf_regression': bug_is_perf_regression,
                           'regressed_perf_tests': regressed_perf_tests_list,
                           'perf_reg_alert_summary_id': perf_reg_alert_summary_id, 
                           'product': bug_product,
                           'component': bug_component,
                           'creation_time': bug_creation_time}
    all_bugs_with_added_regressor_info_list.append(bug_with_added_info)

# for bug in all_bugs_with_added_regressor_info_list:
#     if bug['bug_is_perf_regression'] and bug['bug_is_perf_regressor']:
#         pprint(bug)

all_bugs_with_added_regressor_info_df = pd.DataFrame(all_bugs_with_added_regressor_info_list)
all_bugs_with_added_regressor_info_df.to_csv("../datasets/bugs_with_added_regressor_info.csv", index=False)


43247
43236
43120
43045
42904
42550
1489
1366
316
253
204
190
42272
'regression_bugs_not_in_bugzilla_bugs:\n'
[1325876,
 1665056,
 1883321,
 1388910,
 1769586,
 1594368,
 1800882,
 1874689,
 1878764,
 1878764,
 1878764,
 1878158,
 1887824]

Counter({'Core': 104,
         'Firefox': 17,
         'DevTools': 14,
         'Testing': 13,
         'Firefox Build System': 12,
         'Toolkit': 9,
         'Web Compatibility': 4,
         'Firefox for Android': 3,
         'WebExtensions': 1})
Counter({'Toolchains': 12,
         'JavaScript Engine': 11,
         'Layout': 8,
         'Debugger': 7,
         'Raptor': 7,
         'Graphics: WebRender': 7,
         'Widget: Win32': 6,
         'Graphics': 6,
         'PDF Viewer': 5,
         'Sidebar': 5,
         'Privacy: Anti-Tracking': 5,
         'Interventions': 4,
         'Layout: Text and Fonts': 4,
         'Widget: Gtk': 3,
         'IPC': 3,
         'Netmonitor': 3,
         'CSS Parsing and Computation': 3,
         'General': 